# HyP3 prototype water extent map product

This notebook mimics the workflow a HyP3 Water Map plugin will follow to create an On Demand surface water extent map. It is intended demonstrate *how* HyP3 goes from an On Demand request to a product as well as provide a platform for testing/tuning/refinement ot the water extent map product.

Importantly, step 3 will become the plugin's primary inteface and the options provided to the `make_water_map` function will be the ones provided to users (or a subset thereof). Steps 1 and 2 are required data marshelling steps that will be handled transparently inside of the container.

If you'd like an overview of the water extent mapping process, run the following cell to get the `make_water_map` documentation, otherwise, skip to step 1.

In [ ]:
from asf_tools.water_map import make_water_map

?make_water_map

## 0. Current limitations

* RTC products leverage the [Copernicus GLO-30 Public DEM](https://hyp3-docs.asf.alaska.edu/dems) and have *global* coverage, but we currently need to **manually create HAND data** for each Area of Interest (AIO). We are working on generating global HAND data and will update this notebook when it becomes available. Until then, if you want to look at an area outside the bounds of the HAND source file used in this notebook (see step 2), you can [ask us](https://gitter.im/ASFHyP3/community) or use [hand.py](../asf_tools/hand.py) to generate HAND data over your AIO. 

## 1. Generate input dual-pol RTC product

Water extent mapping requires Radiometrically Terrian Corrected (RTC) Sentinel-1 scenes in order to accurately connect backscatter features to the input HAND data. The next few cells will use HyP3's Python SDK to request an RTC product from HyP3 for the Sentinel-1 granule (scene):

[S1A_IW_GRDH_1SDV_20210413T235641_20210413T235706_037439_0469D0_3F2B](https://search.asf.alaska.edu/#/?zoom=6.964&center=-90.509,40.296&resultsLoaded=true&granule=S1A_IW_GRDH_1SDV_20210413T235641_20210413T235706_037439_0469D0_3F2B-METADATA_GRD_HD&searchType=List%20Search&searchList=S1A_IW_GRDH_1SDV_20210413T235641_20210413T235706_037439_0469D0_3F2B)

### 1.1 Connect to HyP3

In [ ]:
import shutil
import hyp3_sdk as sdk

hyp3 = sdk.HyP3(prompt=True)

### 1.2  Submit an On Demand request for the input RTC product

Currently, the RTC options provided to `hyp3.submit_rtc_job` as specified are expected/required by `make_water_map` and are not reccomended to be changed, except for:

* `speckle_filter` -- Apply an Enhanced Lee speckle filter. 

* `dem_matching` -- Coregisters SAR data to the DEM, rather than using dead reckoning based on orbit files. See the [RTC product guide](https://hyp3-docs.asf.alaska.edu/guides/rtc_product_guide/#terrain-correction) for more information.

* `include_dem` -- include a GeoTIF of the Copernicus DEM pixel aligned to the RTC products. This GeoTIFF isn't used by `make_water_map` but was used to create the RTC product and the HAND data

* `include_rgb` -- include a false-color RGB decomposition for dual-pol granules. This GeoTIFF isn't used by `make_water_map` but may help you [interpret the RTC image](https://github.com/ASFHyP3/hyp3-lib/blob/develop/docs/rgb_decomposition.md#interpreting-the-images) and evalutate the water extent map


In [ ]:
granule = 'S1A_IW_GRDH_1SDV_20210413T235641_20210413T235706_037439_0469D0_3F2B'

job = hyp3.submit_rtc_job(
    granule, name='water-extent-example',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)
print(job)

### 1.3 Wait for the RTC product to be generated

This will take $\approx$ 25 minutes to complete.

In [ ]:
job = hyp3.watch(job)

### 1.4 Download the RTC product

RTC products are downloaded as Zip files and then extracted. The GeoTIF for the VV and VH polarizations are selected from the (unzipped) product folder.

In [ ]:
product_zip = job.download_files()[0]
shutil.unpack_archive(product_zip)
vv_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VV.tif'
vh_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VH.tif'
print(f'VV raster: {vv_raster}')
print(f'VH raster: {vv_raster}')

## 2. Extract HAND data

The HAND image must be pixel-aligned (same extent and size) to the RTC images and is extracted from a hand calculated coverting the input RTC scene.

**Note**: The `hand_source` below was generated mannually to cover an initial Area of Interest (AOI) -- if you want to look at an area outside this AOI, you can [ask us](https://gitter.im/ASFHyP3/community) to generate one over your AOI, or use [hand.py](../asf_tools/hand.py) to generate HAND data yourself. 

In [ ]:
from asf_tools.composite import get_epsg_code
from osgeo import gdal

vh_info = gdal.Info(str(vh_raster), format='json')
vh_epsg = get_epsg_code(vh_info)
vh_bounds = [*vh_info['cornerCoordinates']['upperLeft'], *vh_info['cornerCoordinates']['lowerRight']]

hand_source = '/vsicurl/https://hyp3-testing.s3-us-west-2.amazonaws.com/asf-tools/water-map/mississippi/' \
              'mississippi_hand_cop-glo-30.tif'
hand_raster = str(vh_raster).replace('_VH.tif', '_hand.tif')
gdal.Warp(hand_raster, hand_source, dstSRS=f'EPSG:{vh_epsg}', 
          outputBounds=vh_bounds, width=vh_info['size'][0], height=vh_info['size'][1])

## 3. Generate the water extent map

Now that we have marshalled all the required input data, we can generate the surface water extent map! 

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

from asf_tools.water_map import make_water_map

water_extent_raster = 'water_extent.tif'
make_water_map(water_extent_raster, vv_raster, vh_raster, hand_raster, tile_shape=(100, 100), 
               max_vv_threshold=-17., max_vh_threshold=-24., hand_threshold=15., hand_fraction=0.8)

`make_water_map` in [water_map.py](../asf_tools/water_map.py) is the primary "science" interface for generating the water maps. It *should* include all the necessary options for tuning the product -- if it doesn't, and you'd like other things exposed, either:
* Ask us to expose it by:
  * [Chatting with us in Gitter](https://gitter.im/ASFHyP3/community)
  * [Opening an issue](https://github.com/ASFHyP3/asf-tools/issues/new/choose)
  
* [Fork](https://github.com/ASFHyP3/hyp3-sdk/fork) and propose changes in a Pull Request. See our [contributing guidelines](https://hyp3-docs.asf.alaska.edu/contributing/).

For more info on the `make_water_map` function, run the cell below.

In [ ]:
from asf_tools.water_map import make_water_map
?make_water_map